<a href="https://colab.research.google.com/github/talktokorea/storage/blob/master/lecture4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
#구글 드라이브를 코랩에 마운트 Mount Google Drive
from google.colab import drive     # import drive from google colab
ROOT="/content/drive"                # root directory
drive.mount(ROOT)                     # mount google drive at ROOT= /content/drive
PROJECT_PATH = "/content/drive/MyDrive/Colab Notebooks/storage"
%cd {PROJECT_PATH}
%pwd

In [ ]:
# 1. 기초 데이터 확보
# bream 도미, smelt 빙어 
bream_length = [25.4, 26.3, 26.5, 29.0, 29.0, 29.7, 29.7, 30.0, 30.0, 30.7, 31.0, 31.0, 
                31.5, 32.0, 32.0, 32.0, 33.0, 33.0, 33.5, 33.5, 34.0, 34.0, 34.5, 35.0, 
                35.0, 35.0, 35.0, 36.0, 36.0, 37.0, 38.5, 38.5, 39.5, 41.0, 41.0]
bream_weight = [242.0, 290.0, 340.0, 363.0, 430.0, 450.0, 500.0, 390.0, 450.0, 500.0, 475.0, 500.0, 
                500.0, 340.0, 600.0, 600.0, 700.0, 700.0, 610.0, 650.0, 575.0, 685.0, 620.0, 680.0, 
                700.0, 725.0, 720.0, 714.0, 850.0, 1000.0, 920.0, 955.0, 925.0, 975.0, 950.0]
smelt_length = [9.8, 10.5, 10.6, 11.0, 11.2, 11.3, 11.8, 11.8, 12.0, 12.2, 12.4, 13.0, 14.3, 15.0]
smelt_weight = [6.7, 7.5, 7.0, 9.7, 9.8, 8.7, 10.0, 9.9, 9.8, 12.2, 13.4, 12.2, 19.7, 19.9]
# 2. 입력 데이터 만들기
fish_length = bream_length + smelt_length
fish_weight = bream_weight + smelt_weight
# 넘파이 활용
import numpy as np
fish_input = np.column_stack((fish_length,fish_weight))
print(fish_input[:5])
fish_target = np.concatenate((np.ones(35),np.zeros(14)))
print(fish_target)

In [ ]:
# 3. 훈련 세트와 테스트 세트를 구분(25% 테스트 세트 기본 설정) 
from sklearn.model_selection  import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)
print(train_input.shape, test_input.shape)
print(train_target.shape, test_target.shape)
print(test_target)

In [ ]:
# 3. 훈련 세트와 테스트 세트를 구분(25% 테스트 세트 기본 설정) (계속) 
#stratiy 매개변수에 타겟 전달하여 비율 자동 맞춤(25%)
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, stratify=fish_target,  random_state=42)
print(test_target)
# 4. 산포도 확인
import matplotlib.pyplot as plt
plt.scatter(train_input[:,0],train_input[:,1])
plt.scatter(test_input[:,0],test_input[:,1])
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

In [ ]:
# 5. 알고리즘 적용(동일)
from sklearn.neighbors  import KNeighborsClassifier      # import
kn = KNeighborsClassifier()     # 객체 생성
kn.fit(train_input, train_target)    # 머신러닝 훈련(학습)
print(kn.score(train_input, train_target))  # 정확도(평가)
# 6. 예측 길이 25, 무게 150
kn.predict([[25,150]])

In [ ]:
# 1. 훈련 데이터 대비 특정 생선 포인트(25,150)가 포함된 산포도
import matplotlib.pyplot as plt
#plt.rc('font', family='NanumBarunGothic') # 한글 폰트 인스톨후 런타임 다시 시작
plt.scatter(train_input[:,0],train_input[:,1])
plt.scatter(25,150, marker='*')
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

In [ ]:
# 2. 특정 생선 포인트(25,150)의 이웃 가져오기
distances, indexes = kn.kneighbors([[25,150]])      # distance=이웃거리, indexes=이웃번호
# 이웃 셈플 그리기
plt.scatter(train_input[:,0],train_input[:,1])
plt.scatter(25,150, marker='*')
plt.scatter(train_input[indexes,0],train_input[indexes,1])  # 01번째컬럼 length, 1 2번째컬럼 weight 
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

In [ ]:
# 3. 특정 생선 포인트(25,150)의 이웃 정보 프린트
print(train_input[indexes])
print(train_target[indexes])
print(distances)

In [ ]:
# 4. 기준을 통일하게 1000으로 설정한 후 그리기
plt.scatter(train_input[:,0],train_input[:,1])
plt.scatter(25,150, marker='*')
plt.scatter(train_input[indexes,0],train_input[indexes,1])  # 01번째컬럼 length, 1 2번째컬럼 weight 
plt.xlim((0,1000))
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

In [ ]:
# 5. 표준점수(Z-점수)를 이용한 정규화
mean = np.mean(train_input,axis=0)     # 평균
std = np.std(train_input,axis=0)           # 표준편차
print(mean,std)                                
train_scaled=(train_input-mean)/std    # 학습 데이터 z-value  NumPy 브로드캐스팅

new = ([25,150]-mean)/std      # 생선 포인트 정규화

plt.scatter(train_scaled[:,0], train_scaled[:,1])
plt.scatter(new[0],new[1], marker='*')
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

In [ ]:
# 6. 알고리즘 적용(동일)
from sklearn.neighbors  import KNeighborsClassifier      # import
kn = KNeighborsClassifier()     # 객체 생성
kn.fit(train_scaled,train_target)    # 머신러닝 훈련(학습)
test_scaled = (test_input-mean)/std  # 테스트 데이터 정규화
print(kn.score(test_scaled, test_target))  # 정확도(평가)
# 6. 예측 길이 25, 무게 150
new = ([25,150]-mean)/std      # 생선 포인트 정규화
kn.predict([new])

In [ ]:
# 7. 테스트 데이터의 new 이웃 가져오기
distances, indexes = kn.kneighbors([new])      # distance=이웃거리, indexes=이웃번호
# 이웃 샘플 그리기
plt.scatter(train_scaled[:,0],train_scaled[:,1])
plt.scatter(new[0],new[1], marker='*')
plt.scatter(train_scaled[indexes,0],train_scaled[indexes,1])  # 0=1번째컬럼 length, 1=2번째컬럼 weight 
plt.xlabel('length')
plt.ylabel('weight')
plt.show()
# 이웃 정보 프린트
print(train_scaled[indexes])
print(train_target[indexes])
print(distances)